# Implementación articulo 3D fruit rotation

En este notebook se tiene la implementación tentativa hasta el momento para implementar los puntos del paper 3D fruit rotation.

In [8]:
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt
from Point_311 import pt313utils

import matplotlib.pyplot as plt

from statistics import mean
import math

Se declaran 3 folders - 1 por tipo de fruta (sphere, oblate, prolate)
Se define una lista con estos paths y otra que se usara para definir en que parte del diccionario se guardarán los resultados.

In [9]:
sphere_path = "./data/oranges/obj0001"
oblate_path = "./data/tomatoes/obj0001"
prolate_path = "./data/mandarins/obj0001"    # Fixme: Not sure if mandarin is prolate model

fruits_types = [sphere_path, oblate_path, prolate_path]
fruit_types_by_index = ["sphere", "oblate", "prolate"]

### Diccionario de resultados finales
Este diccionario guarda los resultados hasta el punto 3.1.2 del paper
Se tienen llaves extra para probar las diferentes formulas en los tres tipos de frutas y hacer analisis.

In [10]:
results_dict = {
    "sphere": {
        "semi_axes_a_all": [],
        "semi_axes_b_all": [],
        "principal_axes": {
            "sphere": (0, 0),
            "oblate": (0, 0),
            "prolate": (0, 0)
        }
    },
    "prolate": {
        "semi_axes_a_all": [],
        "semi_axes_b_all": [],
        "principal_axes": {
            "sphere": (0, 0),
            "oblate": (0, 0),
            "prolate": (0, 0)
        }
    },
    "oblate": {
        "semi_axes_a_all": [],  # Major axes
        "semi_axes_b_all": [],  # Minor axes
        "principal_axes": {
            "sphere": (0, 0), # (A, B)
            "oblate": (0, 0), # (A, B)
            "prolate": (0, 0) # (A, B)
        }
    }
}

### Punto 3.1.1 en tipos de frutas
El punto 3.1.3 es para conseguir los semi-ejes "a" y "b" de una vista.
A continuación se muestra un ciclo para conseguir los semi-ejes a y b de todas las vistas de cada tipo de fruta y guardarlos en el diccionario anterior.

__Primer ciclo__
``` for index, fruit_type in enumerate(fruits_types): ```
En este for se ciclan los nombres de los folders con su index.
El indice se ocupa para definir el tipo de fruta actual y guardar los resultados en el diccioanrio correcto
Se crea una lista de todos los nombres de los archivos dentro de folder actual *fruit_type*

__Segundo ciclo__
``` for image_name in image_files: ```
Este ciclo ira por cada vista dentro del folder del tipo de fruta actual y conseguirá los ejes a y b
- Se genera una mascara de la vista actual
- Se calcula la elipse de esa mascara
- Se consigue los ejes (Refers to the full length of the longer and shorter axis of an ellipse)
- Se consiguen los semi-ejes que son la mitad del "full length axe"
- Se guardan los resultados de a y b en el diccionario de resultados


In [4]:
for index, fruit_type in enumerate(fruits_types): # Test with all fruit types
    current_fruit_type_name = fruit_types_by_index[index]
    image_files = [file for file in os.listdir(fruit_type)]  # Get list of all img names inside of folder

    for image_name in image_files:
        file_path = os.path.join(fruit_type, image_name)
        img = cv2.imread(file_path)
        mask = pt313utils.create_mask_from_img(img)
        ellipse = pt313utils.calculate_ellipse_from_mask(mask)
        _, axes_length, _ = ellipse
        minor_axis_length, major_axis_length = axes_length
        semi_major_axis = major_axis_length / 2
        semi_minor_axis = minor_axis_length / 2

        results_dict[current_fruit_type_name]["semi_axes_a_all"].append(semi_major_axis)
        results_dict[current_fruit_type_name]["semi_axes_b_all"].append(semi_minor_axis)

In [5]:
# CHECK RESULTS DICT
print(results_dict["sphere"], "\n")
print(results_dict["oblate"], "\n")
print(results_dict["prolate"], "\n")

{'semi_axes_a_all': [142.6826934814453, 142.5933074951172, 142.96337890625, 143.31948852539062, 142.9832763671875, 143.31939697265625, 142.9354705810547, 141.82737731933594, 141.54840087890625, 143.322998046875, 142.59140014648438, 140.68716430664062, 141.71693420410156, 142.9434051513672, 141.53091430664062, 142.1214141845703], 'semi_axes_b_all': [137.2490997314453, 137.4474639892578, 129.42233276367188, 135.8601837158203, 134.9685516357422, 130.9583740234375, 137.39154052734375, 133.1571807861328, 135.70684814453125, 138.0611572265625, 132.97308349609375, 136.56512451171875, 136.232421875, 131.43499755859375, 137.84100341796875, 137.63406372070312], 'principal_axes': {'sphere': (0, 0), 'oblate': (0, 0), 'prolate': (0, 0)}} 

{'semi_axes_a_all': [157.85122680664062, 156.24771118164062, 158.2526397705078, 155.7021942138672, 156.38575744628906, 156.67051696777344, 157.83145141601562, 158.63279724121094, 159.8738250732422, 162.03494262695312, 156.29812622070312, 156.62393188476562, 157.0

### Punto 3.1.2
Este punto del paper genera la esferoide aproximada usando los ejes de todas las vistas que se consiguen en el punto 3.1.1.
Para cada tipo de fruta (prolate, oblate, sphere) se consigue las 3 esferoides usando cada formula y se guardan el tipo de esferoide aproximada dependiendo la formula que se uso para calcularlo.

In [6]:
# CALCULATE PRINCIPAL AXES WITH ALL FORMULAS TO COMPARE

for index, fruit_type in enumerate(fruit_types_by_index): # Test with all fruit types
    a_all_views = results_dict[fruit_type]["semi_axes_a_all"]
    b_all_views = results_dict[fruit_type]["semi_axes_b_all"]

    # FOR SPHERE FORMULA
    axes_sum = [(x + y) / 2 for x, y in zip(a_all_views, b_all_views)]
    sphere_axe = mean(axes_sum)
    results_dict[fruit_type]["principal_axes"]["sphere"] = (sphere_axe, sphere_axe)

    # FOR OBLATE FORMULA
    results_dict[fruit_type]["principal_axes"]["oblate"] = (mean(a_all_views), min(b_all_views))  # (A, B)

    # FOR PROLATE FORMULA
    results_dict[fruit_type]["principal_axes"]["prolate"] = (max(a_all_views), mean(b_all_views))  # (A, B)

In [ ]:
 # SHOW FOR ONE IMG OF EACH FOLDER THE THREE TYPES OF SPHEROIDS CALCUALTED